# Genie Top200 Chart

In [1]:
import requests
import pandas as pd

#### 1. 인터넷에서 데이터 가져오기

In [2]:
# Genie top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [3]:
# Chrome User-Agent
header = {'User-Agent':
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
req = requests.get(url, headers=header)
html = req.text
#html

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

#### 찾으려고 하는 데이터의 노드 찾기

In [5]:
trs = soup.find_all('tr')
len(trs)

51

In [6]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [7]:
# 개발자 도구에서 찾아주는 Selector
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

#### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
- rank, title, artist, album

In [8]:
tr = trs[0]

In [9]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [10]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

In [11]:
# title
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                취중고백'

In [12]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'취중고백'

In [13]:
# artist
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'김민석 (멜로망스)'

In [14]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'취중고백'

#### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데이터프레임 만들기

In [15]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank) 
    title_list.append(title) 
    artist_list.append(artist) 
    album_list.append(album)


In [16]:
df = pd.DataFrame({
    '순위':rank_list, '제목':title_list, '가수':artist_list, '앨범':album_list
})
df.head()

,순위,제목,가수,앨범
0,1,취중고백,김민석 (멜로망스),취중고백
1,2,겨울잠,아이유 (IU),조각집
2,3,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars


In [17]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank, title, artist, album])

len(lines), len(lines[0])

(50, 4)

In [18]:
df = pd.DataFrame(lines, columns=['순위', '제목', '가수', '앨범'])
df.head()

,순위,제목,가수,앨범
0,1,취중고백,김민석 (멜로망스),취중고백
1,2,겨울잠,아이유 (IU),조각집
2,3,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars


In [19]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d = {'순위':rank, '제목':title, '가수':artist, '앨범':album}
    lines.append(d)

lines[:3]

[{'순위': '1', '제목': '취중고백', '가수': '김민석 (멜로망스)', '앨범': '취중고백'},
 {'순위': '2', '제목': '겨울잠', '가수': '아이유 (IU)', '앨범': '조각집'},
 {'순위': '3',
  '제목': '회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom)',
  '가수': 'sokodomo',
  '앨범': '쇼미더머니 10 Episode 2'}]

In [20]:
df = pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,취중고백,김민석 (멜로망스),취중고백
1,2,겨울잠,아이유 (IU),조각집
2,3,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars


#### 5. 모든 페이지의 데이터 가져오기

In [21]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        lines.append([rank, title, artist, album])

In [22]:
df = pd.DataFrame(lines, columns=['순위', '제목', '가수', '앨범'])
df.tail()

,순위,제목,가수,앨범
195,196,폰서트,10CM,4.0
196,197,Dolphin,오마이걸 (OH MY GIRL),NONSTOP
197,198,Leave The Door Open,Bruno Mars & Anderson .Paak & Silk Sonic,An Evening With Silk Sonic
198,199,시절인연 (時節因緣),이찬원,꼰대인턴 OST Part 2 (MBC 수목드라마)
199,200,사랑이 잘 (With 오혁),아이유 (IU),사랑이 잘


In [23]:
df.to_csv('지니챠트_Top200(211223).csv', index=False)